In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import math
import nolds
from PreprocessFcns import *
%matplotlib inline

In [2]:
full_dict = {}
count = 0
subj_list = ([1004,1016,1018,1019,1020,1024,1029,1030,1032,1038,1044,1046,1047,1049,1051,1052,1053,1054,1055,1056])
for subj in subj_list:
    f = open('Z:\CIS-PD Study\Data_dict\\' + str(subj) + 'dict.pkl', 'rb')
    full_dict[subj] = (pickle.load(f))
    f.close()
    count = count + 1
    if count < 10:
        print(' ' + str(count) + ' of 20 subjects loaded')
    else:
        print(str(count) + ' of 20 subjects loaded')

 1 of 20 subjects loaded
 2 of 20 subjects loaded
 3 of 20 subjects loaded
 4 of 20 subjects loaded
 5 of 20 subjects loaded
 6 of 20 subjects loaded
 7 of 20 subjects loaded
 8 of 20 subjects loaded
 9 of 20 subjects loaded
10 of 20 subjects loaded
11 of 20 subjects loaded
12 of 20 subjects loaded
13 of 20 subjects loaded
14 of 20 subjects loaded
15 of 20 subjects loaded
16 of 20 subjects loaded
17 of 20 subjects loaded
18 of 20 subjects loaded
19 of 20 subjects loaded
20 of 20 subjects loaded


In [3]:
#save corrected version of motor scores.xls 
mot_scores = pd.read_excel('Z:CIS-PD Study\Scores\MotorTasks.xls')
#fix error in original cols labels
mot_scores.columns.values[75] = 'alternating left hand movements tremor right upper limb'
mot_scores.columns.values[150] = 'sitting tremor right upper limb'
#remove words:(Qxx) and 'rating' from each column for readability
cols= mot_scores.columns
cols = cols.tolist()
colsnew = [x.split('(')[0] for x in cols]
colsnew = [x.strip() for x in colsnew] #remove whitespace
colsnew = [x.split('rating')[0] for x in colsnew]
colsnew = [x.strip() for x in colsnew]
colsnew = [x.lower() for x in colsnew] #make all lower case 
colsnew = [x.replace('\x97',' ') for x in colsnew]
colsnew = [x.replace('—',' ') for x in colsnew]    
#simplify notation
for i in range(len(colsnew)):
    x = colsnew[i]
    if x.find('finger to nose')>-1:
        colsnew[i] = x.replace(' hand','')
    if x.find('alternating')>-1:
        colsnew[i] = x.replace(' hand movements','')
c = dict(zip(cols,colsnew))
mot_scores = mot_scores.rename(index=str, columns=c)
#change 1 month label to 4 wks for proper sorting (to incorporate in xls file)
mot_scores.loc[mot_scores['visit']=='1 Month','visit']='4 Weeks'
mot_scores.to_excel('Z:CIS-PD Study\Scores\MotorTasks.xls') #note that timestamps are not properly converted

In [4]:
def DataAggregatorAccelGyro(full_dict,subj_dict,task_list,task_scores,session_list,freq):
    
    Data = pd.DataFrame()
    
    numSamples = pd.DataFrame()
    
    #Calculate mean across all scores for model comparison
    count = 0
    bradyTotal = 0
    tremorTotal = 0
    
    s = 0
    for subj in subj_dict.keys():
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_dict)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            side = subj_dict[subj]
                
            loc = 'dorsal_hand_' + side

            if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                task_score = ts + ' ' + side

            subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb']]
            subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
            subj_score.index = range(len(subj_score))

            n_visits = len(subj_score)
            n_recordings = len(act_dict[task].keys())
            N = n_visits

            if n_visits != n_recordings:
                #print('Recordings error')
                N = min([n_visits,n_recordings])

            for trial in range(N):
                
                sensor = 'accel'
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue

                #ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                #act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                data = HPfilter(act_dict[task][trial][loc][sensor])

                clip_data = gen_clips_mc10(data,downsample=freq,basefreq=62.5)

                feature_extraction(clip_data)
                
                featcols = []

                if 'features' in clip_data.keys():
                    A = clip_data['features']
                    for c in range(len(A.columns)):
                        featcols.append(A.columns[c] + ' acc')
                    
                sensor = 'gyro'
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue
                    
                clip_data = gen_clips_mc10(act_dict[task][trial][loc][sensor],downsample=freq,basefreq=62.5)
                
                feature_extraction(clip_data)
                
                if 'features' in clip_data.keys():
                    G = clip_data['features']
                    for c in range(len(G.columns)):
                        featcols.append(G.columns[c] + ' gyr')
                
                D = pd.concat([A,G],axis=1)
                D.columns = featcols
                D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                D['Tremor'] = subj_score['Tremor'][trial]
                D['Visit'] = session_list[trial]
                D['Task'] = task
                D['Subject'] = subj
                Data = pd.concat([Data,D])
                    
                #clip_lens = []    
                
                #for c in range(len(clip_data['data'])):
                #    clip_lens.append(len(clip_data['data'][c]))
                #N = pd.DataFrame(data=np.asarray(clip_lens),columns=['Samples'],dtype='int')
                #N['Subject'] = subj
                #N['Task'] = task
                #N['Visit'] = session_list[trial]
                #numSamples = pd.concat([numSamples,N])
                                                
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    
    #numSamples = numSamples[['Subject','Visit','Task','Samples']]
    
    #if freq==62.5:
    #    numSamples.to_csv('Z:CIS-PD Study\\Sensor Sample Lengths.csv')
    
    return Data

In [5]:
def DataAggregatorAccelOnly(full_dict,subj_dict,task_list,task_scores,session_list,freq):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    numSamples = pd.DataFrame()
    
    #Calculate mean across all scores for model comparison
    count = 0
    bradyTotal = 0
    tremorTotal = 0
    
    s = 0
    for subj in subj_dict.keys():
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_dict)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            side = subj_dict[subj]
                
            loc = 'dorsal_hand_' + side

            if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
                task_score = ts + ' ' + side

            subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb']]
            subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
            subj_score.index = range(len(subj_score))

            n_visits = len(subj_score)
            n_recordings = len(act_dict[task].keys())
            N = n_visits

            if n_visits != n_recordings:
                #print('Recordings error')
                N = min([n_visits,n_recordings])

            for trial in range(N):
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue

                ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                data = HPfilter(act_dict[task][trial][loc][sensor])

                clip_data = gen_clips_mc10(data,downsample=freq,basefreq=62.5)

                feature_extraction(clip_data)

                if 'features' in clip_data.keys():
                    D = clip_data['features']
                    featcols = list(D.columns)
                    D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                    D['Tremor'] = subj_score['Tremor'][trial]
                    D['Visit'] = session_list[trial]
                    D['Task'] = task
                    D['Subject'] = subj
                    Data = pd.concat([Data,D])
                    
                #clip_lens = []    
                
                #for c in range(len(clip_data['data'])):
                #    clip_lens.append(len(clip_data['data'][c]))
                #N = pd.DataFrame(data=np.asarray(clip_lens),columns=['Samples'],dtype='int')
                #N['Subject'] = subj
                #N['Task'] = task
                #N['Visit'] = session_list[trial]
                #numSamples = pd.concat([numSamples,N])
                                                
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    
    #numSamples = numSamples[['Subject','Visit','Task','Samples']]
    
    #if freq==62.5:
    #    numSamples.to_csv('Z:CIS-PD Study\\Sensor Sample Lengths.csv')
    
    return Data

In [6]:
subj_dict = {1004:'right',1016:'left',1018:'left',1019:'left',1020:'right',1024:'left',1029:'left',1030:'left',1032:'left',
            1038:'left',1044:'right',1046:'right',1049:'left',1051:'left'}
task_list = ['Motor #1: Standing','Motor #2: Walking', 'Motor #3: Walking while Counting','Motor #4: Finger to Nose',
             'Motor #5: Alternating Hand Movements','Motor #6: Sit to Stand','Motor #7: Drawing on Paper',
             'Motor #8: Typing on a Computer','Motor #9: Nuts and Bolts','Motor #10: Drinking Water',
             'Motor #11: Organizing Folder','Motor #12: Folding Towels','Motor #13: Sitting']
task_scores = ['standing','walking','walking while counting','finger to nose','alternating','sit to stand',
               'drawing on a paper','typing on a computer keyboard','assembling nuts and bolts',
               'taking a glass of water and drinking','organizing sheets in a folder','folding towels','sitting']
session_list = ['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', '2 Weeks: Time 120', 
               '2 Weeks: Time 150', '4 Weeks']

In [7]:
#for freq in [62.5,60,55,50,45,40,35,30,25,20,15,10,5]:
for freq in [7.5,3]:
    print(freq)
    Data = DataAggregatorAccelGyro(full_dict,subj_dict,task_list,task_scores,session_list,freq)
    Data.to_csv('Z:CIS-PD Study\\Downsample\\Accel+Gyro Data ' + str(freq) +  'Hz.csv')
    Data = DataAggregatorAccelOnly(full_dict,subj_dict,task_list,task_scores,session_list,freq)
    Data.to_csv('Z:CIS-PD Study\\Downsample\\Accel Data ' + str(freq) +  'Hz.csv')

7.5
Subject 1004 (1 of 14)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
Subject 1004 (1 of 14)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
3
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)


In [10]:
freq = 62.5
print(freq)
Data = DataAggregatorAccelOnly(full_dict,subj_dict,task_list,task_scores,session_list,freq)
Data.to_csv('Z:CIS-PD Study\\Sensor Data No Resample.csv')

62.5
Subject 1004 (1 of 14)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)


In [29]:
Data.columns

Index(['Subject', 'Visit', 'Task', 'Bradykinesia', 'Tremor', 'RMSX_acc',
       'RMSY_acc', 'RMSZ_acc', 'rangeX_acc', 'rangeY_acc', 'rangeZ_acc',
       'meanX_acc', 'meanY_acc', 'meanZ_acc', 'varX_acc', 'varY_acc',
       'varZ_acc', 'skewX_acc', 'skewY_acc', 'skewZ_acc', 'kurtX_acc',
       'kurtY_acc', 'kurtZ_acc', 'xcor_peakXY_acc', 'xcorr_peakXZ_acc',
       'xcorr_peakYZ_acc', 'xcorr_lagXY_acc', 'xcorr_lagXZ_acc',
       'xcorr_lagYZ_acc', 'Dom_freq_acc', 'Pdom_rel_acc', 'PSD_mean_acc',
       'PSD_std_acc', 'PSD_skew_acc', 'PSD_kur_acc', 'jerk_mean_acc',
       'jerk_std_acc', 'jerk_skew_acc', 'jerk_kur_acc', 'Sen_X_acc',
       'Sen_Y_acc', 'Sen_Z_acc', 'RMS_mag_acc', 'range_mag_acc',
       'mean_mag_acc', 'var_mag_acc', 'skew_mag_acc', 'kurt_mag_acc',
       'Sen_mag_acc', 'RMSX gyr', 'RMSY gyr', 'RMSZ gyr', 'rangeX gyr',
       'rangeY gyr', 'rangeZ gyr', 'meanX gyr', 'meanY gyr', 'meanZ gyr',
       'varX gyr', 'varY gyr', 'varZ gyr', 'skewX gyr', 'skewY gyr',
       'skewZ

In [30]:
pd.read_csv('Z:CIS-PD Study\\Sensor Data Resample 62.5HZ.csv')

,Unnamed: 0,Subject,Visit,Task,Bradykinesia,Tremor,RMSX,RMSY,RMSZ,rangeX,...,PSD_std,PSD_skew,PSD_kur,jerk_mean,jerk_std,jerk_skew,jerk_kur,Sen_X,Sen_Y,Sen_Z
0,0,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000732,0.000535,0.000574,0.079929,...,2.248566e-05,3.957944,17.453930,-4.201932e-05,0.004184,-0.066907,0.425589,0.895181,1.171897,0.991753
1,1,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000591,0.000492,0.000475,0.059164,...,1.399018e-05,1.614171,1.790243,-4.759900e-05,0.004053,-0.367448,1.388203,0.952226,0.906698,1.149938
2,2,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000557,0.000494,0.000422,0.063106,...,3.637439e-06,1.311572,0.411762,2.628928e-06,0.003752,-0.306528,1.186797,0.890457,0.892128,1.325102
3,3,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000472,0.000360,0.000383,0.056094,...,5.820581e-06,1.514496,1.703693,1.555026e-05,0.003292,0.132470,0.111247,1.015921,1.299535,1.247401
4,4,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000317,0.000244,0.000281,0.033473,...,4.003085e-06,2.311033,5.007486,4.050112e-05,0.002590,0.053671,0.837272,1.289370,1.392917,1.263504
5,5,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000212,0.000133,0.000164,0.023385,...,8.652582e-07,1.570975,2.379631,-1.363500e-05,0.002120,0.145974,-0.105358,1.508184,1.839541,1.900959
6,6,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000300,0.000183,0.000163,0.052590,...,7.610505e-07,2.491438,5.976862,1.335471e-05,0.002163,0.067363,0.325810,1.261292,1.396683,1.971345
7,7,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000382,0.000188,0.000178,0.065765,...,1.216245e-06,1.811544,2.483537,3.085347e-05,0.002284,-0.106633,0.139448,0.997457,1.333003,1.897487
8,8,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000319,0.000140,0.000178,0.045893,...,1.486284e-06,2.521806,6.361772,-1.067269e-05,0.002178,-0.074210,-0.255324,1.242819,1.703678,1.831201
9,9,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000216,0.000145,0.000167,0.019202,...,1.302053e-06,2.918695,8.016578,-3.619097e-07,0.002062,0.023303,-0.285454,1.611734,1.560382,1.882209


In [31]:
pd.read_csv('Z:CIS-PD Study\\Sensor Data No Resample.csv')

,Unnamed: 0,Subject,Visit,Task,Bradykinesia,Tremor,RMSX,RMSY,RMSZ,rangeX,...,PSD_std,PSD_skew,PSD_kur,jerk_mean,jerk_std,jerk_skew,jerk_kur,Sen_X,Sen_Y,Sen_Z
0,0,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000732,0.000535,0.000574,0.079929,...,2.245687e-05,3.957944,17.453930,-4.201932e-05,0.004184,-0.066907,0.425589,0.895181,1.171897,0.991753
1,1,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000591,0.000492,0.000475,0.059164,...,1.397226e-05,1.614171,1.790243,-4.759900e-05,0.004053,-0.367448,1.388203,0.952226,0.906698,1.149938
2,2,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000557,0.000494,0.000422,0.063106,...,3.632781e-06,1.311572,0.411762,2.628928e-06,0.003752,-0.306528,1.186797,0.890457,0.892128,1.325102
3,3,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000472,0.000360,0.000383,0.056094,...,5.813128e-06,1.514496,1.703693,1.555026e-05,0.003292,0.132470,0.111247,1.015921,1.299535,1.247401
4,4,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000317,0.000244,0.000281,0.033473,...,3.997959e-06,2.311033,5.007486,4.050112e-05,0.002590,0.053671,0.837272,1.289370,1.392917,1.263504
5,5,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000212,0.000133,0.000164,0.023385,...,8.641504e-07,1.570975,2.379631,-1.363500e-05,0.002120,0.145974,-0.105358,1.508184,1.839541,1.900959
6,6,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000300,0.000183,0.000163,0.052590,...,7.600760e-07,2.491438,5.976862,1.335471e-05,0.002163,0.067363,0.325810,1.261292,1.396683,1.971345
7,7,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000382,0.000188,0.000178,0.065765,...,1.214683e-06,1.811544,2.483537,3.085347e-05,0.002284,-0.106633,0.139448,0.997457,1.333003,1.897487
8,8,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000319,0.000140,0.000178,0.045893,...,1.484381e-06,2.521806,6.361772,-1.067269e-05,0.002178,-0.074210,-0.255324,1.242819,1.703678,1.831201
9,9,1004,2 Weeks: Time 0,Motor #1: Standing,NaN,0.0,0.000216,0.000145,0.000167,0.019202,...,1.300386e-06,2.918695,8.016578,-3.619097e-07,0.002062,0.023303,-0.285454,1.611734,1.560382,1.882209


In [68]:
pd.read_csv('Z:CIS-PD Study\\Sensor Sample Lengths.csv',index_col=0)

,Subject,Visit,Task,Samples
0,1004,2 Weeks: Time 0,Motor #1: Standing,313
1,1004,2 Weeks: Time 0,Motor #1: Standing,313
2,1004,2 Weeks: Time 0,Motor #1: Standing,313
3,1004,2 Weeks: Time 0,Motor #1: Standing,313
4,1004,2 Weeks: Time 0,Motor #1: Standing,313
5,1004,2 Weeks: Time 0,Motor #1: Standing,313
6,1004,2 Weeks: Time 0,Motor #1: Standing,313
7,1004,2 Weeks: Time 0,Motor #1: Standing,312
8,1004,2 Weeks: Time 0,Motor #1: Standing,313
9,1004,2 Weeks: Time 0,Motor #1: Standing,313


In [ ]:
f = open('Z:\CIS-PD Study\Data_dict\\' + str(1004) + 'dict.pkl', 'rb')
act_dict = (pickle.load(f))
f.close()

In [ ]:
rawdata = act_dict['Motor #1: Standing'][0]['dorsal_hand_left']['accel']
clip_data = gen_clips_mc10(rawdata)
feature_extraction(clip_data)
clip_data['features'].columns